In [8]:
import json
import pandas as pd

from google.oauth2 import service_account

# Lire les données depuis BigQuery

In [9]:
with open("ensai-2024-630e074aa45c.json") as source:
    info = json.load(source)

credentials = service_account.Credentials.from_service_account_info(info)

prenoms = pd.read_gbq("ml.prenoms", project_id="ensai-2024", credentials=credentials)

In [10]:
prenoms.head()

,sexe,preusuel,annais,dpt,nombre
0,1,_PRENOMS_RARES,1900,13,37
1,1,_PRENOMS_RARES,1900,57,37
2,1,_PRENOMS_RARES,1900,59,44
3,1,_PRENOMS_RARES,1900,62,34
4,1,_PRENOMS_RARES,1900,75,45


# Pre-processing

1. Exclure les "_PRENOMS_RARES"
2. Aggréger les "annais" et les "dpt" pour avoir la fréquence par preusuel
3. Déterminer le genre d'un prénom (optionnel)

In [55]:
preprocessing = prenoms[prenoms["preusuel"] != '_PRENOMS_RARES']
preprocessing["nombre"] = preprocessing["nombre"].astype(int)
preprocessing = preprocessing.groupby(["preusuel", "sexe"]).agg({"nombre": sum}).reset_index()

/var/folders/rs/7bw0gmgj5hqfyg0fpk1d13t00000gn/T/ipykernel_62004/570584931.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessing["nombre"] = preprocessing["nombre"].astype(int)


In [56]:
preprocessing.head()

,preusuel,sexe,nombre
0,A,1,27
1,AADAM,1,32
2,AADEL,1,57
3,AADHIRA,2,22
4,AADIL,1,190


In [62]:
data = (
    preprocessing
        .iloc[preprocessing.groupby("preusuel")["nombre"].idxmax()]
        .drop(["nombre"], axis=1)
)

In [63]:
data.head()

,preusuel,sexe
0,A,1
1,AADAM,1
2,AADEL,1
3,AADHIRA,2
4,AADIL,1
